In [ ]:
import pickle
import random
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
class MeanEmbeddingVectorizer(object):

    def __init__(self, word_model,pre_trained):
        self.pre_trained = pre_trained
        self.word_model = word_model
        if self.pre_trained == False:
            self.vector_size = word_model.wv.vector_size
        else:
            self.vector_size = word_model.vector_size

    def fit(self):  # comply with scikit-learn transformer requirement
        return self

    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector

    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.


        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """
        mean = []
        for word in sent:
            if self.pre_trained == False:
                if word in self.word_model.wv.vocab:
                    mean.append(self.word_model.wv.get_vector(word))
            else:
                if word in self.word_model.vocab:
                    mean.append(self.word_model.get_vector(word))
        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            #logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.

        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])

In [ ]:
def predict(text):

  clf = pickle.load(open('/content/best_model_svm.sav', 'rb'))

  vocab = KeyedVectors.load_word2vec_format(open('/content/GoogleNews-vectors-negative300.bin.gz', 'rb'), binary=False)
  w2v_mean =MeanEmbeddingVectorizer(vocab,pre_trained=True)

  rl_test_x = []
  rl_test_x.append(gensim.utils.simple_preprocess(text))


  rl_test_x = w2v_mean.transform(rl_test_x)
  y_pred = clf.predict(rl_test_x)

predict("A strong start nearly saw the visitors go ahead after 45 seconds when Ben Woodburn poked just wide from Liam Millar's cross. But they wouldn't have to wait too long for the opener to arrive.")

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.23.0 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


UnicodeDecodeError: ignored